In [1]:
import numpy as np
import pandas as pd
import random
import torch
import pickle

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
from PortfolioConstructor import PortfolioConstructor
from ExchnageEnv import MarketEnvironment

In [4]:
if torch.cuda.is_available():
    torch.set_default_device('cuda') 
    torch.get_default_device()
    device = 'cuda'
    
else:
    device = 'cpu'

print(f"device : {device}")

device : cuda


In [5]:
pkl_fpath = '/home/naradaw/dev/Charles_Schwab/data/historical/2024_09_11/historical_price_seq_2024_09_11_12_04.pkl'

with open(pkl_fpath, 'rb') as f:
    price_sqs_dict = pickle.load(f)

In [6]:
price_sqs_dict['A'].shape

(1174, 61)

In [7]:
test_symbol_uni = random.choices(list(price_sqs_dict.keys()), k = 20)

In [8]:
portfolio_constructor = PortfolioConstructor(
    device= device,
    symbol_universe = test_symbol_uni,
    seq_length = 60,
    multihead_dim = 2,
    num_transformer_layers = 2
)

/home/naradaw/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
market_env = MarketEnvironment(
    data_path = pkl_fpath,
    holding_period = 1,
    train_test_split= 0.8,
    symbol_universe = test_symbol_uni,
    device = device
    )

In [10]:
market_env.data.shape

(1174, 20, 61)

# Utility

In [11]:
''' 
sharpe ratio measures the excess return of the portfolio over the 
volatility of it -> risk adjusted performance
'''


def sharp_ratio_(rewards, tran_costs):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

# Train

In [12]:
portfolio_constructor.cuda()
portfolio_constructor.train()
market_env.reset(mode = "train")

In [13]:
portfolio_constructor.parameters()

<generator object Module.parameters at 0x7f1f8cb925f0>

In [14]:
optimizer = torch.optim.Adam(portfolio_constructor.parameters())

In [15]:
market_env.get_state()

tensor([[171.4500, 168.8400, 179.2800,  ..., 207.2000, 208.0400, 208.1200],
        [159.6700, 159.5300, 161.6500,  ..., 171.2300, 170.0400, 167.6300],
        [ 86.9400,  86.4700,  89.8700,  ..., 106.9000, 106.9400, 108.3600],
        ...,
        [ 52.5600,  52.2300,  54.0000,  ...,  50.3600,  50.7000,  50.9600],
        [203.8900, 203.9700, 206.5600,  ..., 235.3400, 233.5800, 233.7100],
        [ 66.1000,  65.1000,  69.4900,  ..., 128.8500, 127.9300, 127.6700]],
       device='cuda:0')

# Sandbox

In [16]:
def sharp_ratio_loss_(rewards, tran_costs, allocations):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

In [17]:
training_steps = 500
eval_step = 32
train_step = 32


In [18]:
def evaluate(model, env):
    model.eval()
    is_end = False
    rewards = []
    tran_costs = []
    
    env.reset(mode = "test")
    state = env.get_state()

    while not is_end:
        symbol_idx , allocations = model(state)
        state, reward, is_end, tran_cost = env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)

    # print()
    # print("evaluation")
    # print(f"rewards : {rewards}")
    # print(f"tran_costs : {tran_costs}")
    # print()

    sharp_ratio = sharp_ratio_(rewards, tran_costs)
    
    model.train()

    return sharp_ratio

In [19]:
writer = SummaryWriter()

In [20]:
max_reward = -1

for training_step in range(training_steps):
    is_end = False
    rewards = []
    tran_costs = []
    nlls = []

    market_env.reset(mode = "train", transaction_cost= 1e-7)
    state = market_env.get_state()

    while not is_end:
        symbol_idx, allocations = portfolio_constructor(state)
        state, reward, is_end, tran_cost = market_env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)
        mask_tensor = torch.tensor([1 if i in symbol_idx.cpu().numpy() else 0 for i in range(allocations.shape[0])]).type(torch.FloatTensor).cuda()
        nlls.append(torch.log(allocations.abs() / 2 + 1e-9) * mask_tensor)

    sharp_ratio = sharp_ratio_(rewards, tran_costs)
    loss = -sharp_ratio * sum([e.sum() for e in nlls])

    loss.backward(retain_graph=True)

    if (training_step + 1) % train_step == 0:

        print("-------------------------------------")
        print("training model")
        print('Step {}: last loss = {:.5f}\r'.format(training_step, loss), end='')
        print()
        writer.add_scalar("Loss/train", sharp_ratio, training_step)
        # pprint([(n, e.grad) for n, e in model.named_parameters()])
        optimizer.step()
        optimizer.zero_grad()
        count = 0
        
    if (training_step + 1) % eval_step == 0:
        with torch.no_grad():
            reward_val = evaluate(portfolio_constructor, market_env)
            print('Step {}: val_rewards = {}'.format(training_step, reward_val))
            writer.add_scalar("eval_sharpe/train", reward_val, training_step)

            if max_reward < reward_val:
                max_reward = reward_val

                print("*** found better model ***")
            print()
                # torch.save(portfolio_constructor.state_dict(), model_path)




-------------------------------------
training model
Step 31: last loss = 46.68164
Step 31: val_rewards = 0.5545058859363626
*** found better model ***

-------------------------------------
training model
Step 63: last loss = 150.25212
Step 63: val_rewards = 0.44946712964972263

-------------------------------------
training model
Step 95: last loss = 202.76048
Step 95: val_rewards = 0.5241538101517977

-------------------------------------
training model
Step 127: last loss = -69215191040.00000
Step 127: val_rewards = 0.51488741242285

-------------------------------------
training model
Step 159: last loss = -21.22056
Step 159: val_rewards = 0.711848088555702
*** found better model ***

-------------------------------------
training model
Step 191: last loss = 121.14175
Step 191: val_rewards = 0.6389666811215293



In [21]:
random_state = market_env.get_random_state()
random_state.detach().cpu().numpy()

array([[257.77, 253.37, 254.9 , ..., 319.88, 314.37, 304.23],
       [ 43.93,  43.56,  44.08, ...,  46.11,  46.07,  46.48],
       [116.19, 113.36, 113.92, ..., 149.9 , 151.73, 150.02],
       ...,
       [ 62.16,  61.94,  65.92, ...,  77.11,  75.18,  74.31],
       [ 50.42,  50.07,  51.14, ...,  53.87,  53.97,  51.78],
       [ 29.66,  29.95,  30.84, ...,  34.78,  34.7 ,  34.4 ]],
      dtype=float32)

In [22]:
portfolio_constructor(random_state)

(tensor([ 1,  2,  4,  6,  7, 11, 14, 15, 18, 19], device='cuda:0'),
 tensor([0.0000, 0.1080, 0.0973, 0.0000, 0.0972, 0.0000, 0.0994, 0.0980, 0.0000,
         0.0000, 0.0000, 0.1090, 0.0000, 0.0000, 0.0976, 0.0975, 0.0000, 0.0000,
         0.0982, 0.0978], device='cuda:0', grad_fn=<SoftmaxBackward0>))

In [23]:
writer.add_graph(portfolio_constructor, random_state.detach())

/home/naradaw/dev/Charles_Schwab/code/RnD/enviroment/PortfolioConstructor.py:142: TracerWarning: torch.Tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  long_mask = torch.Tensor([0 if i in long_sqs else 1 for i in range(rank.shape[0])]).to(self.device)
/home/naradaw/dev/Charles_Schwab/code/RnD/enviroment/PortfolioConstructor.py:170: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  portfolio_allocations = [allocation.item() for allocation in allocations if allocation != 0]


In [24]:
writer.flush()
writer.close()

In [ ]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
